<a href="https://colab.research.google.com/github/adityasengar/Hoffman_games/blob/main/Hoffman_Games.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
The simulation consists of a 2D world where initially some humans are present.
Each human is a conscious agent (humans react differently when alone and when in presence of a fellow human).
I start the simulation at time t=0 when only humans are present and then perform calculations at future time steps.
At each time step, the following happens:
- A human can move left, right, up, down or stay where they are.
- If a human sees another human nearby, the tendency of both humans to stay at their respective location increases.
  This condition resembles a process of cooperation between human beings to ensure maximal survival.
- Whenever 2 humans are nearby, they have a chance to construct a machine which is placed randomly in the near vicinity of the human pair.
- If a human sees a machine nearby, it has a chance to fuse with the machine to become AI.
- Humans have a natural birth rate (by reproduction), and a natural death rate.
  Apart from this, they are depleted when AI is formed
- Machines are formed when humans interact and are depleted when AI is formed.
- AI are formed when humans and machine combine.
  AI have a natural death rate  much smaller than the death rate of humans.

'''


import random
import numpy as np
import math

#helping functions here
#initialise matrix Mat with the agents. Also create a matrix P that contains the relative probability of an agent to not move
def init(a):

    f=open("matrix_vs_t.dat","w")
    f1 = open("density_vs_t.dat", "w")

    start=[]
    c=0
    for i in range(len(a)):
        for j in range(len(a[0])):
            rand=random.uniform(0, 1)
            if(rand<p_dense):
                a[i][j]=1
                P[i][j]=1
                c=c+1
                #c=c+1
                start.append((i,j))
            f.write(str(a[i][j])+"\t")
        f.write("\n")
    return (start,a,c)

def timer(a,i1,j1):
    num1 = 1  # move left. keep at 1
    num2 = 1  # move right. keep at 1
    num3 = 1  # move up. keep at 1
    num4 = 1  # move down. keep at 1
    rand = random.uniform(0, 1)
    num5 = P[i1][j1]  # move nowhere
    numtot = num1 + num2 + num3 + num4 + num5
    rand1 = num1 / numtot
    rand2 = rand1 + num2 / numtot
    rand3 = rand2 + num3 / numtot
    rand4 = rand3 + num4 / numtot
    rand5 = rand4 + num5 / numtot
    # 0-0.25:move left,  0.25-0.5:move right, 0.5-0.75-move up, 0.75-1.0:move up
    if (rand < rand1):
        if (a[(i1 - 1) % len(a)][j1] == 0):
            a[(i1 - 1) % len(a)][j1] = a[i1][j1] * fader
            a[i1][j1] = 0
            P[(i1 - 1) % len(a)][j1] = P[i1][j1]
            P[i1][j1] = 0
            num[j] = ((i1 - 1) % len(a), j1)
    elif (rand < rand2):
        if (a[(i1 + 1) % len(a)][j1] == 0):
            a[(i1 + 1) % len(a)][j1] = a[i1][j1] * fader

            a[i1][j1] = 0
            P[(i1 + 1) % len(a)][j1] = P[i1][j1]
            P[i1][j1] = 0
            num[j] = ((i1 + 1) % len(a), j1)
    elif (rand < rand3):
        if (a[i1][(j1 + 1) % len(a[0])] == 0):
            a[i1][(j1 + 1) % len(a[0])] = a[i1][j1] * fader
            a[i1][j1] = 0
            P[i1][(j1 + 1) % len(a[0])] = P[i1][j1]
            P[i1][j1] = 0
            num[j] = (i1, (j1 + 1) % len(a[0]))
    elif (rand < rand4):
        if (a[i1][(j1 - 1) % len(a[0])] == 0):
            a[i1][(j1 - 1) % len(a[0])] = a[i1][j1] * fader
            a[i1][(j1 - 1) % len(a[0])] = a[i1][j1] * fader
            a[i1][j1] = 0
            P[i1][(j1 - 1) % len(a[0])] = P[i1][j1]
            P[i1][j1] = 0
            num[j] = (i1, (j1 - 1) % len(a[0]))

    return 0


#function to determine the dynamics of a machine and AI
def ai(a,i,j,cman,cmachine,cai):
    probi=k_aicreate
    rand = random.uniform(0, 1)
    if (rand < probi):
        if a[(i - 1) % dim][j] == 0.5:
            if(a[i][j]==1):
                cman=cman-1
                cai=cai+1
            a[i][j] = 0.25
            a[(i - 1) % dim][j] = 0
            cmachine=cmachine-1
            return (cman,cmachine,cai)
        if a[(i + 1) % dim][j] == 0.5:
            if (a[i][j] == 1):
                cman = cman - 1
                cai = cai + 1
            cmachine=cmachine-1
            a[i][j] = 0.25
            a[(i + 1) % dim][j] = 0
            return (cman,cmachine,cai)
        if a[i][(j - 1) % dim] == 0.5:
            if (a[i][j] == 1):
                cman = cman - 1
                cai = cai + 1
            cmachine=cmachine-1
            a[i][j] = 0.25
            a[i][(j - 1) % dim] = 0
            return (cman,cmachine,cai)
        if a[i][(j + 1) % dim] == 0.5:
            if (a[i][j] == 1):
                cman = cman - 1
                cai = cai + 1
            cmachine=cmachine-1
            a[i][j] = 0.25
            a[i][(j + 1) % dim] = 0
            return (cman,cmachine,cai)
    return (cman,cmachine,cai)


#neighbor interactions estimator
def neighbour(a,i,j,c):
    qq=c
    if a[(i - 1) % dim][j]==a[i][j] or a[(i + 1) % dim][j]==a[i][j] or a[i][(j + 1) % dim]==a[i][j] or a[i][(j - 1) % dim]==a[i][j]:
        if (a[i][j] == 1):
            P[i][j] = 1000.0
        if (a[i][j] == 0.25):
            P[i][j] = 10.0
        qq = computer_create(a, i, j,qq)

        if a[(i - 1) % dim][j] == 0 and a[(i + 1) % dim][j] == 0 and a[i][(j + 1) % dim] == 0 and a[i][
            (j - 1) % dim] == 0:
            P[i][j] = 1.0

    return qq

#Creation of a new computer
def computer_create(a,i,j,c):
    qq=c
    list1=[(i-3)%dim, (i-2)%dim, (i-1)%dim, (i+1)%dim, (i+2)%dim,(i+3)%dim]
    list2=[(j-3)%dim, (j-2)%dim, (j-1)%dim, (j+1)%dim, (j+2)%dim,(j+3)%dim]
    random.shuffle(list1)
    random.shuffle(list2)
    for i1 in range( len(list1) ):
        for j1 in range ( len(list2)):
            rand = random.uniform(0, 1)
            if a[list1[i1]][list2[j1]]==0 and rand<k_compcreate:
                a[list1[i1]][list2[j1]]=0.5
                qq=qq+1
                return qq
    return qq

#birth of man, and death of machine
def birth(a,cman,cai,cmachine):
    for i in range(dim):
        for j in range(dim):
            if (a[i][j] == 0):
                rand = random.uniform(0, 1)
                pdense2 = k_birth * ((cman / (dim * dim)))
                pdense3= k_birth* (((cman+cai) / (dim * dim)))

                if (rand < pdense2):
                    a[i][j] = 1
                    P[i][j] = 1
                    cman = cman + 1
                    num.append((i, j))
                elif (rand < pdense3):
                    a[i][j] = 0.25
                    P[i][j] = 1
                    cai = cai + 1
                    num.append((i, j))
            elif(a[i][j]==0.5):
                rand=random.uniform(0,1)
                pdense4= k_deathmachine
                if rand<pdense4:
                    a[i][j]=0
                    cmachine=cmachine-1

    return (cman,cai,cmachine)

#death of man and ai
def death(a,cman,cai):
    j1=0
    while j1<len(num):
        i=num[j1][0]
        j=num[j1][1]
        if (a[i][j] == 1):
            rand = random.uniform(0, 1)
            pdeath =  k_death
            if (rand < pdeath):
                a[i][j] = 0
                P[i][j] = 0
                cman = cman - 1
                num.pop(j1)
                j1=j1-1
        elif (a[i][j] == 0.25):
            rand = random.uniform(0, 1)
            pdeath =  k_deathai
            if (rand < pdeath):
                a[i][j] = 0
                P[i][j] = 0
                cai = cai - 1
                num.pop(j1)
                j1 = j1 - 1

        j1=j1+1
    return (cman,cai)

#variables defined here
k_aicreate=0.0003
p_dense=0.02
k_birth=0.3 #at this parameter, naturally population doubles evevry 1500 time steps. (all other rates 0)
k_death=0.2 #at this parameter, population doubles in 5000 time steps. Beware of to.
k_deathai=0.2
k_deathmachine=0.1
k_compcreate=0.00005
to=100#time interval at which birth and death activated



dim=50
time=100000
Mat= [[0 for k in range(dim)] for j in range(dim)]
P =  [[0 for j in range(dim)] for i in range(dim)]
fader=1
cman=0
cmachine=0
cai=0

#Main function callings here
#num returns initial position of players. Mat returns the matrix with position of players
num,Mat,cman=init(Mat) #initialise matrix m at time t=0
#print(cman)
f = open("matrix_vs_t.dat", "a")
f1 = open("density_vs_t.dat", "a")

for t in range(time - 1):
    random.shuffle(num)
    # print(num)
    if (t%to==0):
        #print(len(num))
        (cman,cai,cmachine)=birth(Mat,cman,cai,cmachine)
       # print(len(num))
        (cman,cai)=death(Mat,cman,cai)
        #print(len(num))
        #print("*")


    for j in range(len(num)):
        i1 = num[j][0]
        j1 = num[j][1]
        timer(Mat, i1,j1)
        cmachine = neighbour(Mat,i1,j1,cmachine) # finds if neighbour is human, and creates computer
        #print(cmachine)
        (cman,cmachine,cai)=ai(Mat,i1,j1,cman,cmachine,cai)



    f1.write(str(t)+"\t"+str(cman/(dim*dim))+"\t"+str(cmachine/(dim*dim))+"\t"+str(cai/(dim*dim)))
    f1.write("\n")

    if (t%10==0):
        for i in range(len(Mat[0])):
            for j in range(len(Mat[0])):
                f.write(str(Mat[i][j]) + "\t")
            f.write("\n")




